In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install -q --upgrade transformers datasets evaluate sacrebleu sentencepiece scikit-learn


In [1]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [2]:
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer
)
import torch
import numpy as np

MODEL_NAME = "Helsinki-NLP/opus-mt-en-fr"
OUTPUT_DIR = "./translation_model"
BATCH_SIZE = 8
MAX_INPUT_LENGTH = 64
MAX_TARGET_LENGTH = 64
PREFIX = "translate English to French: "

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Running on:", device)


2025-10-24 12:46:34.500432: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761309994.684491     104 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761309994.734828     104 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Running on: cuda


I am just using 5% of dataset otherwise the training times are insane

In [3]:
print("Loading")
raw_datasets = load_dataset("opus_books", "en-fr")
full_train = raw_datasets["train"]
sample_size = max(1, int(0.05 * len(full_train)))
val_split = int(0.9 * sample_size)
train = full_train.select(range(val_split))
val = full_train.select(range(val_split, sample_size))

print(f"Train size: {len(train)}, Validation size: {len(val)})")

Loading


README.md: 0.00B [00:00, ?B/s]

en-fr/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/127085 [00:00<?, ? examples/s]

Train size: 5718, Validation size: 636)


In [4]:

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

def preprocess_examples(examples):
    inputs = [PREFIX + t["en"] for t in examples["translation"]]
    targets = [t["fr"] for t in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=MAX_INPUT_LENGTH, truncation=True)
    labels = tokenizer(text_target=targets, max_length=MAX_TARGET_LENGTH, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

print("Tokenizing datasets")
train_tokens = train.map(preprocess_examples, batched=True)
val_tokens = val.map(preprocess_examples, batched=True)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

source.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

Tokenizing datasets


model.safetensors:   0%|          | 0.00/301M [00:00<?, ?B/s]

Map:   0%|          | 0/5718 [00:00<?, ? examples/s]

Map:   0%|          | 0/636 [00:00<?, ? examples/s]

In [5]:
from transformers import EarlyStoppingCallback

In [6]:
import evaluate
bleu = evaluate.load("sacrebleu")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels

def compute_metrics(eval_pred):
    preds, labels = eval_pred
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    result = bleu.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

training_args = Seq2SeqTrainingArguments(
    output_dir=OUTPUT_DIR,
    eval_strategy="epoch",
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    predict_with_generate=True,
    logging_dir=f"{OUTPUT_DIR}/logs",
    save_total_limit=2,
    num_train_epochs=10,
    fp16=False,
    save_strategy="epoch",
    learning_rate=5e-5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_tokens,
    eval_dataset=val_tokens,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
    
)


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipykernel_104/2207071198.py:37: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [7]:
trainer.train()
trainer.save_model(OUTPUT_DIR)
print("Model trained da!:")


The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.


Epoch,Training Loss,Validation Loss,Bleu
1,2.032800,2.233086,12.906101
2,1.671600,2.251130,13.370427
3,1.198700,2.262501,13.601515


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[59513]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.encoder.embed_positions.weight', 'model.decoder.embed_tokens.weight', 'model.decoder.embed_positions.weight', 'lm_head.weight'].


Model trained da!:


In [8]:

from sklearn.metrics import precision_recall_fscore_support
preds, refs = [], []

for example in val:
    src = PREFIX + example["translation"]["en"]
    input_ids = tokenizer(src, return_tensors="pt").input_ids.to(model.device)
    output_ids = model.generate(input_ids, max_length=MAX_TARGET_LENGTH, num_beams=4)
    pred = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    preds.append(pred)
    refs.append(example["translation"]["fr"])

# BLEU
bleu = evaluate.load("sacrebleu")
bleu_result = bleu.compute(predictions=preds, references=[[r] for r in refs])
print(f"\nBLEU score: {bleu_result['score']:.2f}")



BLEU score: 11.03


obviously the score is kinda low coz i just used 5% of dataset, but i beleive otherwise the models and methods are solid tho